In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [2]:
import re
import stanza
import pickle
import datasets
from collections import Counter
import dictionary_definition
stanza.download('en')  
from transformers import AutoModelWithLMHead, AutoTokenizer

2020-05-01 00:35:29 INFO: Downloading default packages for language: en (English)...
2020-05-01 00:35:30 INFO: File exists: /home/tdimson/stanza_resources/en/default.zip.
2020-05-01 00:35:33 INFO: Finished downloading models and saved to /home/tdimson/stanza_resources.


In [14]:
blacklist = (
    datasets.Blacklist(set())
    .merge(datasets.Blacklist.from_text_stream(open("../data/wikitext-103-raw/wiki.train.raw"), sample_rate=0.05, use_gpu=True))
    .merge(datasets.Blacklist.from_text_stream(open("../data/wikitext-103-raw/wiki.test.raw"),  use_gpu=True))
    .merge(datasets.Blacklist.from_text_lines(open("../data/words.txt")))
    .merge(datasets.Blacklist.from_parsed_dictionary("../data/en_dictionary_parsed_randomized.pickle"))
)

2020-05-01 00:40:11 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-05-01 00:40:11 INFO: Use device: gpu
2020-05-01 00:40:11 INFO: Loading: tokenize
2020-05-01 00:40:11 INFO: Done loading processors!
2020-05-01 00:40:36 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-05-01 00:40:36 INFO: Use device: gpu
2020-05-01 00:40:36 INFO: Loading: tokenize
2020-05-01 00:40:36 INFO: Done loading processors!
2020-05-01 00:41:00 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-05-01 00:41:00 INFO: Use device: gpu
2020-05-01 00:41:00 INFO: Loading: tokenize
2020-05-01 00:41:00 INFO: Done loading processors!
2020-05-01 00:41:25 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     

293872


2020-05-01 00:49:29 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-05-01 00:49:29 INFO: Use device: gpu
2020-05-01 00:49:29 INFO: Loading: tokenize
2020-05-01 00:49:29 INFO: Done loading processors!


19460


In [18]:
len(blacklist.blacklist_set)

767691

In [17]:
blacklist.merge(datasets.Blacklist.from_text_lines(open("../data/words.txt")))

In [19]:
blacklist.dump("../data/blacklist.pickle")

In [3]:
parsed_dict = pickle.load(open("../data/en_dictionary_parsed_randomized.pickle", "rb"))

In [ ]:
has_period = 0
no_period = 0
c = Counter()
for item in parsed_dict[:1000]:
    for sense in item.senses:
        for definition in sense.definitions:
            if isinstance(definition, dictionary_definition.ReferenceDefinition):
                continue
                
            c[definition.definition.strip()[-1]] += 1
            if definition.definition.strip().endswith("."):
                has_period += 1
            else:
                no_period += 1
                
print(f"Has period: {100 * has_period / (has_period + no_period):.2f}")

In [14]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
ip = datasets.InverseParsedDictionaryDefinitionDataset(tokenizer, None, None)

In [20]:
for item in parsed_dict[:100]:
    for i, sense in enumerate(item.senses):
        for definition in sense.definitions:
            if isinstance(definition, dictionary_definition.ReferenceDefinition):
                continue
            
            if definition.definition.strip()[-1] == ".":
                e = ip._make_examples(tokenizer, item)[i]
                if e:
                    print(tokenizer.decode(e))

<|bod|> a protein present in the blood, from which the substance of egg yolk is derived <|bd|> vitellogenin <|pos|> noun <|eod|>
<|bod|> a method of dating rocks from the relative proportions of radioactive potassium-40 and its decay product, argon-40 <|bd|> potassium–argon dating <|pos|> noun <|eod|>
<|bod|> (in Vedanta) any of the three interdependent modes or qualities of prakriti: sattva, rajas, or tamas <|bd|> guna <|pos|> noun <|eod|>
<|bod|> waterproof or water-resistant clothes suitable for wearing in the rain <|bd|> rainwear <|pos|> noun <|eod|>
<|bod|> make a serious or urgent request, typically to the public <|bd|> appeal <|pos|> verb <|be|> police are appealing for information about the incident <|be|> she appealed to Germany for political asylum <|eod|>
<|bod|> a person who installs and repairs the pipes and fittings of water supply, sanitation, or heating systems <|bd|> plumber <|pos|> noun <|eod|>
<|bod|> (of a syllable) without accent or stress <|bd|> atonic <|pos|> adj

In [ ]:
_)